In [6]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3, DenseNet121, MobileNetV2, NASNetLarge
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Constants
IMG_SIZE = 224  # Depending on the architecture, you might need to adjust this
BATCH_SIZE = 32
EPOCHS = 20
DATA_DIR = 'data\\Data_png'
NUM_CLASSES = 3
VALIDATION_SPLIT = 0.2

# Fetch all files and labels
file_paths = []
labels = []
class_folders = ['MR_MFCC', 'MS_MFCC', 'Normal_MFCC']

for folder in class_folders:
    folder_path = os.path.join(DATA_DIR, folder)
    for file in os.listdir(folder_path):
        file_paths.append(os.path.join(folder_path, file))
        labels.append(folder)

# Split the data into training and validation sets
train_files, val_files, train_labels, val_labels = train_test_split(file_paths, labels, test_size=VALIDATION_SPLIT)

# Data Generators
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(dataframe=pd.DataFrame({'filename': train_files, 'class': train_labels}),
                                                    x_col="filename",
                                                    y_col="class",
                                                    target_size=(IMG_SIZE, IMG_SIZE),
                                                    batch_size=BATCH_SIZE,
                                                    class_mode='categorical')

validation_generator = test_datagen.flow_from_dataframe(dataframe=pd.DataFrame({'filename': val_files, 'class': val_labels}),
                                                        x_col="filename",
                                                        y_col="class",
                                                        target_size=(IMG_SIZE, IMG_SIZE),
                                                        batch_size=BATCH_SIZE,
                                                        class_mode='categorical')

# Architectures and training as before...


Found 383 validated image filenames belonging to 3 classes.
Found 97 validated image filenames belonging to 3 classes.


C:\Users\Admin\miniconda1\envs\GPU_mini\lib\site-packages\keras\preprocessing\image.py:1139: UserWarning: Found 3 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


In [4]:
# Architectures
def get_model(base_model):
    # Freezing Layers
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(NUM_CLASSES, activation='softmax')(x)

    return Model(inputs=base_model.input, outputs=predictions)

# Models
MODELS = {
    "VGG16": VGG16,
    "ResNet50": ResNet50,
    "InceptionV3": InceptionV3,
    "DenseNet121": DenseNet121,
    "MobileNetV2": MobileNetV2,
    "NASNetLarge": NASNetLarge
}

for name, model in MODELS.items():
    print(f"Training model: {name}")

    base = model(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
    model = get_model(base)

    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(train_generator,
              validation_data=validation_generator,
              epochs=EPOCHS,
              steps_per_epoch=len(train_generator),
              validation_steps=len(validation_generator))

    print(f"{name} Model Training Completed!")

Training model: VGG16
Epoch 1/20
12/12 [==============================] - 4s 289ms/step - loss: 1.1183 - accuracy: 0.3473 - val_loss: 1.1124 - val_accuracy: 0.2990
Epoch 2/20
12/12 [==============================] - 3s 281ms/step - loss: 1.0820 - accuracy: 0.3446 - val_loss: 1.0748 - val_accuracy: 0.2990
Epoch 3/20
12/12 [==============================] - 3s 282ms/step - loss: 1.0680 - accuracy: 0.4726 - val_loss: 1.0595 - val_accuracy: 0.3093
Epoch 4/20
12/12 [==============================] - 3s 287ms/step - loss: 1.0534 - accuracy: 0.5822 - val_loss: 1.0367 - val_accuracy: 0.6289
Epoch 5/20
12/12 [==============================] - 4s 304ms/step - loss: 1.0397 - accuracy: 0.6397 - val_loss: 1.0286 - val_accuracy: 0.7216
Epoch 6/20
12/12 [==============================] - 3s 281ms/step - loss: 1.0235 - accuracy: 0.7337 - val_loss: 1.0075 - val_accuracy: 0.8866
Epoch 7/20
12/12 [==============================] - 4s 305ms/step - loss: 1.0098 - accuracy: 0.7415 - val_loss: 0.9950 - val_a

KeyboardInterrupt: 

In [ ]:
import tensorflow as tf

# Architectures
def get_model(base_model):
    # Freezing Layers
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(NUM_CLASSES, activation='softmax')(x)

    return Model(inputs=base_model.input, outputs=predictions)

# Models
MODELS = {
    "VGG16": VGG16,
    "ResNet50": ResNet50,
    "InceptionV3": InceptionV3,
    "DenseNet121": DenseNet121,
    "MobileNetV2": MobileNetV2,
    "NASNetLarge": NASNetLarge
}


if tf.test.gpu_device_name():
    print("Default GPU Device: {}".format(tf.test.gpu_device_name()))
    for name, model in MODELS.items():
        print(f"Training model: {name}")

        base = model(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
        model = get_model(base)

        model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

        model.fit(train_generator,
                  validation_data=validation_generator,
                  epochs=EPOCHS,
                  steps_per_epoch=len(train_generator),
                  validation_steps=len(validation_generator))

        print(f"{name} Model Training Completed!")
else:
    print("Please install GPU version of TensorFlow")
    for name, model in MODELS.items():
        print(f"Training model: {name}")

        base = model(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
        model = get_model(base)

        model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

        model.fit(train_generator,
                  validation_data=validation_generator,
                  epochs=EPOCHS,
                  steps_per_epoch=len(train_generator),
                  validation_steps=len(validation_generator))

        print(f"{name} Model Training Completed!")



12/12 [==============================] - 7s 595ms/step - loss: 0.2794 - accuracy: 0.9401 - val_loss: 0.2298 - val_accuracy: 0.9479
Epoch 114/1000
12/12 [==============================] - 8s 659ms/step - loss: 0.2786 - accuracy: 0.9401 - val_loss: 0.2172 - val_accuracy: 0.9583
Epoch 115/1000
12/12 [==============================] - 7s 614ms/step - loss: 0.2626 - accuracy: 0.9557 - val_loss: 0.2293 - val_accuracy: 0.9479
Epoch 116/1000
12/12 [==============================] - 7s 614ms/step - loss: 0.2789 - accuracy: 0.9375 - val_loss: 0.2123 - val_accuracy: 0.9583
Epoch 117/1000
12/12 [==============================] - 7s 614ms/step - loss: 0.2659 - accuracy: 0.9531 - val_loss: 0.2293 - val_accuracy: 0.9479
Epoch 118/1000
12/12 [==============================] - 7s 615ms/step - loss: 0.2741 - accuracy: 0.9401 - val_loss: 0.2092 - val_accuracy: 0.9583
Epoch 119/1000
12/12 [==============================] - 7s 621ms/step - loss: 0.2665 - accuracy: 0.9583 - val_loss: 0.2165 - val_accuracy: 

12/12 [==============================] - 3s 277ms/step - loss: 0.1970 - accuracy: 0.9479 - val_loss: 0.1594 - val_accuracy: 0.9688
Epoch 170/1000
12/12 [==============================] - 3s 277ms/step - loss: 0.1954 - accuracy: 0.9401 - val_loss: 0.1446 - val_accuracy: 0.9583
Epoch 171/1000
12/12 [==============================] - 3s 281ms/step - loss: 0.1873 - accuracy: 0.9583 - val_loss: 0.1467 - val_accuracy: 0.9583
Epoch 172/1000
12/12 [==============================] - 3s 277ms/step - loss: 0.1959 - accuracy: 0.9557 - val_loss: 0.1506 - val_accuracy: 0.9688
Epoch 173/1000
12/12 [==============================] - 3s 279ms/step - loss: 0.1947 - accuracy: 0.9557 - val_loss: 0.1419 - val_accuracy: 0.9583
Epoch 174/1000
12/12 [==============================] - 3s 276ms/step - loss: 0.1781 - accuracy: 0.9557 - val_loss: 0.1470 - val_accuracy: 0.9583
Epoch 175/1000
12/12 [==============================] - 3s 279ms/step - loss: 0.1795 - accuracy: 0.9583 - val_loss: 0.1443 - val_accuracy: 

12/12 [==============================] - 3s 276ms/step - loss: 0.1523 - accuracy: 0.9609 - val_loss: 0.1110 - val_accuracy: 0.9792
Epoch 226/1000
12/12 [==============================] - 3s 277ms/step - loss: 0.1491 - accuracy: 0.9635 - val_loss: 0.1271 - val_accuracy: 0.9688
Epoch 227/1000
12/12 [==============================] - 3s 277ms/step - loss: 0.1517 - accuracy: 0.9609 - val_loss: 0.1074 - val_accuracy: 0.9792
Epoch 228/1000
12/12 [==============================] - 3s 278ms/step - loss: 0.1547 - accuracy: 0.9583 - val_loss: 0.1210 - val_accuracy: 0.9688
Epoch 229/1000
12/12 [==============================] - 3s 279ms/step - loss: 0.1522 - accuracy: 0.9479 - val_loss: 0.1105 - val_accuracy: 0.9792
Epoch 230/1000
12/12 [==============================] - 3s 284ms/step - loss: 0.1628 - accuracy: 0.9531 - val_loss: 0.1091 - val_accuracy: 0.9688
Epoch 231/1000
12/12 [==============================] - 3s 280ms/step - loss: 0.1592 - accuracy: 0.9531 - val_loss: 0.1061 - val_accuracy: 

12/12 [==============================] - 3s 277ms/step - loss: 0.1339 - accuracy: 0.9661 - val_loss: 0.0921 - val_accuracy: 0.9792
Epoch 282/1000
12/12 [==============================] - 3s 279ms/step - loss: 0.1288 - accuracy: 0.9688 - val_loss: 0.0893 - val_accuracy: 0.9792
Epoch 283/1000
12/12 [==============================] - 3s 279ms/step - loss: 0.1289 - accuracy: 0.9635 - val_loss: 0.0944 - val_accuracy: 0.9792
Epoch 284/1000
12/12 [==============================] - 3s 276ms/step - loss: 0.1188 - accuracy: 0.9688 - val_loss: 0.0903 - val_accuracy: 0.9792
Epoch 285/1000
12/12 [==============================] - 3s 277ms/step - loss: 0.1277 - accuracy: 0.9661 - val_loss: 0.0901 - val_accuracy: 0.9792
Epoch 286/1000
12/12 [==============================] - 3s 276ms/step - loss: 0.1126 - accuracy: 0.9740 - val_loss: 0.0912 - val_accuracy: 0.9792
Epoch 287/1000
12/12 [==============================] - 3s 274ms/step - loss: 0.1206 - accuracy: 0.9688 - val_loss: 0.0880 - val_accuracy: 

12/12 [==============================] - 3s 279ms/step - loss: 0.1224 - accuracy: 0.9688 - val_loss: 0.0772 - val_accuracy: 0.9792
Epoch 338/1000
12/12 [==============================] - 3s 277ms/step - loss: 0.1229 - accuracy: 0.9661 - val_loss: 0.0805 - val_accuracy: 0.9896
Epoch 339/1000
12/12 [==============================] - 3s 281ms/step - loss: 0.1028 - accuracy: 0.9714 - val_loss: 0.0771 - val_accuracy: 0.9792
Epoch 340/1000
12/12 [==============================] - 3s 276ms/step - loss: 0.0948 - accuracy: 0.9792 - val_loss: 0.0769 - val_accuracy: 0.9792
Epoch 341/1000
12/12 [==============================] - 3s 276ms/step - loss: 0.1234 - accuracy: 0.9635 - val_loss: 0.0808 - val_accuracy: 0.9792
Epoch 342/1000
12/12 [==============================] - 3s 276ms/step - loss: 0.1124 - accuracy: 0.9661 - val_loss: 0.0816 - val_accuracy: 0.9792
Epoch 343/1000
12/12 [==============================] - 3s 279ms/step - loss: 0.1099 - accuracy: 0.9740 - val_loss: 0.0749 - val_accuracy: 

...

12/12 [==============================] - 3s 274ms/step - loss: 0.0741 - accuracy: 0.9818 - val_loss: 0.0628 - val_accuracy: 0.9896
Epoch 450/1000
12/12 [==============================] - 3s 277ms/step - loss: 0.0812 - accuracy: 0.9792 - val_loss: 0.0558 - val_accuracy: 0.9896
Epoch 451/1000
12/12 [==============================] - 3s 274ms/step - loss: 0.0815 - accuracy: 0.9714 - val_loss: 0.0669 - val_accuracy: 0.9792
Epoch 452/1000
12/12 [==============================] - 3s 278ms/step - loss: 0.0917 - accuracy: 0.9766 - val_loss: 0.0629 - val_accuracy: 0.9896
Epoch 453/1000
12/12 [==============================] - 3s 273ms/step - loss: 0.0727 - accuracy: 0.9740 - val_loss: 0.0584 - val_accuracy: 0.9896
Epoch 454/1000
12/12 [==============================] - 3s 274ms/step - loss: 0.0759 - accuracy: 0.9818 - val_loss: 0.0662 - val_accuracy: 0.9792
Epoch 455/1000
12/12 [==============================] - 3s 281ms/step - loss: 0.0892 - accuracy: 0.9714 - val_loss: 0.0588 - val_accuracy: 

12/12 [==============================] - 3s 280ms/step - loss: 0.0884 - accuracy: 0.9740 - val_loss: 0.0546 - val_accuracy: 0.9896
Epoch 506/1000
12/12 [==============================] - 3s 278ms/step - loss: 0.0769 - accuracy: 0.9870 - val_loss: 0.0524 - val_accuracy: 0.9896
Epoch 507/1000
12/12 [==============================] - 3s 276ms/step - loss: 0.0651 - accuracy: 0.9844 - val_loss: 0.0559 - val_accuracy: 0.9896
Epoch 508/1000
12/12 [==============================] - 3s 275ms/step - loss: 0.0835 - accuracy: 0.9792 - val_loss: 0.0561 - val_accuracy: 0.9896
Epoch 509/1000
12/12 [==============================] - 3s 279ms/step - loss: 0.0707 - accuracy: 0.9766 - val_loss: 0.0553 - val_accuracy: 0.9896
Epoch 510/1000
12/12 [==============================] - 3s 280ms/step - loss: 0.0930 - accuracy: 0.9766 - val_loss: 0.0540 - val_accuracy: 0.9896
Epoch 511/1000
12/12 [==============================] - 3s 277ms/step - loss: 0.0716 - accuracy: 0.9844 - val_loss: 0.0562 - val_accuracy: 

12/12 [==============================] - 3s 281ms/step - loss: 0.0629 - accuracy: 0.9870 - val_loss: 0.0542 - val_accuracy: 0.9896
Epoch 562/1000
12/12 [==============================] - 3s 274ms/step - loss: 0.0691 - accuracy: 0.9818 - val_loss: 0.0483 - val_accuracy: 0.9896
Epoch 563/1000
12/12 [==============================] - 3s 276ms/step - loss: 0.0787 - accuracy: 0.9792 - val_loss: 0.0567 - val_accuracy: 0.9896
Epoch 564/1000
12/12 [==============================] - 3s 273ms/step - loss: 0.0691 - accuracy: 0.9792 - val_loss: 0.0531 - val_accuracy: 0.9896
Epoch 565/1000
12/12 [==============================] - 3s 277ms/step - loss: 0.0635 - accuracy: 0.9792 - val_loss: 0.0477 - val_accuracy: 0.9896
Epoch 566/1000
12/12 [==============================] - 3s 277ms/step - loss: 0.0525 - accuracy: 0.9948 - val_loss: 0.0478 - val_accuracy: 0.9896
Epoch 567/1000
12/12 [==============================] - 3s 275ms/step - loss: 0.0694 - accuracy: 0.9844 - val_loss: 0.0493 - val_accuracy: 

12/12 [==============================] - 3s 276ms/step - loss: 0.0730 - accuracy: 0.9714 - val_loss: 0.0572 - val_accuracy: 0.9896
Epoch 618/1000
12/12 [==============================] - 3s 275ms/step - loss: 0.0575 - accuracy: 0.9896 - val_loss: 0.0437 - val_accuracy: 0.9896
Epoch 619/1000
12/12 [==============================] - 3s 279ms/step - loss: 0.0615 - accuracy: 0.9922 - val_loss: 0.0461 - val_accuracy: 0.9896
Epoch 620/1000
12/12 [==============================] - 3s 277ms/step - loss: 0.0783 - accuracy: 0.9661 - val_loss: 0.0526 - val_accuracy: 0.9896
Epoch 621/1000
12/12 [==============================] - 3s 278ms/step - loss: 0.0648 - accuracy: 0.9740 - val_loss: 0.0425 - val_accuracy: 0.9896
Epoch 622/1000
12/12 [==============================] - 3s 278ms/step - loss: 0.0590 - accuracy: 0.9870 - val_loss: 0.0447 - val_accuracy: 0.9896
Epoch 623/1000
12/12 [==============================] - 3s 271ms/step - loss: 0.0631 - accuracy: 0.9792 - val_loss: 0.0467 - val_accuracy: 

12/12 [==============================] - 3s 279ms/step - loss: 0.0504 - accuracy: 0.9896 - val_loss: 0.0425 - val_accuracy: 1.0000
Epoch 674/1000
12/12 [==============================] - 3s 292ms/step - loss: 0.0548 - accuracy: 0.9870 - val_loss: 0.0407 - val_accuracy: 1.0000
Epoch 675/1000
12/12 [==============================] - 3s 285ms/step - loss: 0.0601 - accuracy: 0.9870 - val_loss: 0.0473 - val_accuracy: 1.0000
Epoch 676/1000
12/12 [==============================] - 3s 280ms/step - loss: 0.0642 - accuracy: 0.9818 - val_loss: 0.0448 - val_accuracy: 0.9896
Epoch 677/1000
12/12 [==============================] - 3s 278ms/step - loss: 0.0611 - accuracy: 0.9922 - val_loss: 0.0388 - val_accuracy: 0.9896
Epoch 678/1000
12/12 [==============================] - 3s 276ms/step - loss: 0.0423 - accuracy: 0.9922 - val_loss: 0.0428 - val_accuracy: 1.0000
Epoch 679/1000
 1/12 [=>............................] - ETA: 3s - loss: 0.0198 - accuracy: 1.0000

In [ ]:
# ... [previous imports]

import matplotlib.pyplot as plt

# ... [previous constants and data generators]

# Define a dictionary to store results
results = {}

for name, model_func in MODELS.items():
    print(f"Training model: {name}")

    base = model_func(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
    model = get_model(base)

    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

    history = model.fit(train_generator,
                        validation_data=validation_generator,
                        epochs=EPOCHS,
                        steps_per_epoch=len(train_generator),
                        validation_steps=len(validation_generator))

    # Save results
    results[name] = history.history

    print(f"{name} Model Training Completed!")

# Plot results
colors = ['r', 'g', 'b', 'y', 'm', 'c']

plt.figure(figsize=(14, 6))

# Plotting accuracy
plt.subplot(1, 2, 1)
for name, color in zip(MODELS.keys(), colors):
    plt.plot(results[name]['accuracy'], color=color, label=f"{name} Train")
    plt.plot(results[name]['val_accuracy'], '--', color=color, label=f"{name} Val")
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()

# Plotting loss
plt.subplot(1, 2, 2)
for name, color in zip(MODELS.keys(), colors):
    plt.plot(results[name]['loss'], color=color, label=f"{name} Train")
    plt.plot(results[name]['val_loss'], '--', color=color, label=f"{name} Val")
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()

plt.tight_layout()
plt.show()
